In [1]:
import numpy as np 
import pandas as pd 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('mitbih_database'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

mitbih_database\100.csv
mitbih_database\100annotations.txt
mitbih_database\101.csv
mitbih_database\101annotations.txt
mitbih_database\102.csv
mitbih_database\102annotations.txt
mitbih_database\103.csv
mitbih_database\103annotations.txt
mitbih_database\104.csv
mitbih_database\104annotations.txt
mitbih_database\105.csv
mitbih_database\105annotations.txt
mitbih_database\106.csv
mitbih_database\106annotations.txt
mitbih_database\107.csv
mitbih_database\107annotations.txt
mitbih_database\108.csv
mitbih_database\108annotations.txt
mitbih_database\109.csv
mitbih_database\109annotations.txt
mitbih_database\111.csv
mitbih_database\111annotations.txt
mitbih_database\112.csv
mitbih_database\112annotations.txt
mitbih_database\113.csv
mitbih_database\113annotations.txt
mitbih_database\114.csv
mitbih_database\114annotations.txt
mitbih_database\115.csv
mitbih_database\115annotations.txt
mitbih_database\116.csv
mitbih_database\116annotations.txt
mitbih_database\117.csv
mitbih_database\117annotations.t

In [2]:
import matplotlib.pyplot as plt
import csv
import pywt
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

import os

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, Flatten, Dense, Dropout, Softmax, MaxPooling1D, AveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import regularizers


In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [8]:
plt.rcParams["figure.figsize"] = (10,6)

In [9]:
classes = ['N', 'L', 'R', 'A', 'V']
n_classes = len(classes)
count_classes = [0]*n_classes
count_classes

[0, 0, 0, 0, 0]

In [10]:
path = '/kaggle/mitbih-database'
filenames = next(os.walk(path))[2]
records=list()
annotations=list()
filenames.sort()

StopIteration: 

In [ ]:
for f in filenames:
    filename, file_extension = os.path.splitext(f)
    
    if(file_extension=='.csv'):
        records.append(path+'/'+filename+file_extension)
    else:
        annotations.append(path+'/'+filename+file_extension)

records[0]

In [ ]:
temp_signal=[]
with open(records[6],'r') as csvfile:
    filereader = csv.reader(csvfile,delimiter=',',quotechar='|')
    row_index = -1
    for row in filereader:
        if(row_index >= 0):
            temp_signal.insert(row_index, int(row[1]))
        row_index += 1

In [ ]:
temp_signal[:10]

In [ ]:
plt.plot(temp_signal[:700])
plt.xlabel('Samples', fontsize=15)
plt.ylabel('Voltage (Millivolts)', fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('before_preprocessing')

In [ ]:
temp2signal=temp_signal

In [ ]:
def denoise(data): 
    w = pywt.Wavelet('sym4')
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.04 # Threshold for filtering

    coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
  #  print(len(coeffs))
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))
        
    datarec = pywt.waverec(coeffs, 'sym4')
    return datarec

In [ ]:
temp_signal=denoise(temp_signal)
plt.plot(temp_signal[:700])
plt.xlabel('Samples', fontsize=15)
plt.ylabel('Voltage (Millivolts)', fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('after_denoising')

In [ ]:
temp_signal = stats.zscore(temp_signal)
plt.plot(temp_signal[:700])
plt.xlabel('Samples', fontsize=15)
plt.ylabel('Voltage (Millivolts)', fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('after_norm')

# Preprocessing: Denoising, Scaling, Classes Segmentation


In [ ]:
X=list()
y=list()
window_size=180
for r in range(0,len(records)):
    signals = []

    with open(records[r], 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|') 
        row_index = -1
        for row in spamreader:
            if(row_index >= 0):
                signals.insert(row_index, int(row[1]))
            row_index += 1

        
    signals = denoise(signals)
    #signals = signals.reshape(-1,1)
    signals = stats.zscore(signals)

    with open(annotations[r], 'r') as fileID:
        data = fileID.readlines() 
        beat = list()

        for d in range(1, len(data)): 
            splitted = data[d].split(' ') 
            splitted = filter(None, splitted)
            next(splitted) 
            pos = int(next(splitted)) 
            arrhythmia_type = next(splitted) 
            
            if(arrhythmia_type in classes):
                arrhythmia_index = classes.index(arrhythmia_type)
                count_classes[arrhythmia_index] += 1
                if(window_size <= pos and pos < (len(signals) - window_size)):
                    beat = signals[pos-window_size:pos+window_size]   

                    X.append(beat)
                    y.append(arrhythmia_index)

# data shape
print(np.shape(X), np.shape(y))

In [ ]:
count_classes

In [ ]:
i=0;
while y[i]!=0:
    i=i+1

plt.plot(X[i])
plt.title('N (Normal beat) ', fontsize=20)
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.savefig('N')

In [ ]:
y_df = pd.DataFrame(y)
y_df.head()
per_class = y_df[y_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(30,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['N', 'L', 'R', 'V', 'A'], colors=['#2085ec','#72b4eb','#0a417a','#8464a0','#cea9bc'],autopct='%1.1f%%', textprops={'fontsize':15})
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()
p.savefig('Before_piechart')

In [ ]:
y_df = pd.DataFrame(y)
y_df.head()
per_class = y_df[y_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(30,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['N', 'L', 'R', 'V', 'A'], colors=['#2085ec','#72b4eb','#0a417a','#8464a0','#cea9bc'],autopct='%1.1f%%', textprops={'fontsize':15})
plt.show()
plt.savefig('Before_piechart')

In [ ]:
X = np.array(X)
X_reshaped = X.reshape(-1,360,)
X_reshaped.shape

In [ ]:
X_df = pd.DataFrame(X_reshaped)
X_df.head()

In [ ]:
X_new_df = pd.concat([X_df,y_df],axis=1)
X_new_df.head()

In [ ]:
ax=list(range(361))
X_new_df = X_new_df.set_axis(ax, axis='columns')

In [ ]:
df_0=(X_new_df[X_new_df[X_new_df.shape[1]-1]==0]).sample(n=20000,random_state=42)
df_1=X_new_df[X_new_df[X_new_df.shape[1]-1]==1]
df_2=X_new_df[X_new_df[X_new_df.shape[1]-1]==2]
df_3=X_new_df[X_new_df[X_new_df.shape[1]-1]==3]
df_4=X_new_df[X_new_df[X_new_df.shape[1]-1]==4]

df_1_upsample=resample(df_1,replace=True,n_samples=7000,random_state=125)
df_2_upsample=resample(df_2,replace=True,n_samples=7000,random_state=77)
df_3_upsample=resample(df_3,replace=True,n_samples=7000,random_state=103)
df_4_upsample=resample(df_4,replace=True,n_samples=7000,random_state=59)

X_new_df=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])


In [ ]:
per_class = X_new_df[X_new_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(30,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['N', 'L', 'R', 'V', 'A'], colors=['#2085ec','#72b4eb','#0a417a','#8464a0','#cea9bc'],autopct='%1.1f%%',textprops={'fontsize':15})
plt.show()
plt.savefig('after_piechart')

In [ ]:
i=0;
while y[i]!=0:
    i=i+1

plt.plot(X[i])
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('N')

In [ ]:
i=0;
while y[i]!=1:
    i=i+1

plt.plot(X[i])
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('L')

In [ ]:
i=0;
while y[i]!=2:
    i=i+1

plt.plot(X[i]) 
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('R')

In [ ]:
i=0;
while y[i]!=3:
    i=i+1

plt.plot(X[i])
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('A')

In [ ]:
i=0;
while y[i]!=4:
    i=i+1

plt.plot(X[i])
plt.xlabel('samples',fontsize=15);
plt.ylabel('Voltage (Millivolts)',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('V')

In [ ]:
train, test = train_test_split(X_new_df, test_size=0.20, random_state=7)

print("X_train : ", np.shape(train))
print("X_test  : ", np.shape(test))

In [ ]:
target_train=train[train.shape[1]-1]
target_test=test[test.shape[1]-1]
y_train=to_categorical(target_train)
y_test=to_categorical(target_test)
print(np.shape(y_train), np.shape(y_test))

In [ ]:
X_train = train.iloc[:,:train.shape[1]-1].values
X_test = test.iloc[:,:test.shape[1]-1].values
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)
print(np.shape(X_train), np.shape(X_test))

In [ ]:
# model = Sequential()
# model.add(Conv1D(filters=16, kernel_size=11, padding='same', activation='gelu',input_shape=(360, 1)))
# model.add(MaxPooling1D(pool_size=3, strides=2))
# model.add(Conv1D(filters=32, kernel_size=13, padding='same', activation='gelu'))
# model.add(MaxPooling1D(pool_size=3, strides=2))
# model.add(Conv1D(filters=64, kernel_size=15, padding='same', activation='gelu'))
# model.add(MaxPooling1D(pool_size=3, strides=2))
# model.add(Conv1D(filters=128, kernel_size=17, padding='same', activation='gelu'))
# model.add(MaxPooling1D(pool_size=3, strides=2))
# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(70,kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
# model.add(Dense(35,kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
# model.add(Dense(5,kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
# model.add(Softmax())
# model.summary()


model = Sequential()

model.add(Convolution1D(filters=16, kernel_size=11, strides=1, padding='same' , activation = 'relu', input_shape=(360,1)))
model.add(MaxPooling1D(pool_size=5, strides=2))




model.add(Convolution1D(filters=32, kernel_size=13, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))



model.add(Convolution1D(filters=64, kernel_size=15, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))


model.add(Convolution1D(filters=128, kernel_size=17, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))



model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))

model.add(Dense(50, activation='relu'))

model.add(Dense(5, activation='softmax'))
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=30, epochs=60, verbose=1, validation_data=(X_test, y_test))

In [ ]:
val_loss, val_accuracy = model.evaluate(X_test, y_test, batch_size=30, verbose=1)


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy',fontsize=20)
plt.xticks(fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel('accuracy',fontsize=15)
plt.xlabel('epoch',fontsize=15)
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('accuracy_plot')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss', fontsize=20)
plt.ylabel('loss', fontsize=15)
plt.xlabel('epoch', fontsize=15)
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('loss_plot')
plt.show()


In [ ]:
score = model.evaluate(X_test, y_test)

print('Test Loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)
ax=plt.subplot()
custCnnConfMat = confusion_matrix(y_true, prediction)
sns.heatmap(custCnnConfMat, annot=True,fmt='d', cmap='Greens',ax=ax)
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(classes); ax.yaxis.set_ticklabels(classes);
plt.savefig('cm')

In [ ]:
from sklearn.metrics import classification_report

cf = classification_report(y_true, prediction, target_names=classes,digits=4)
print(cf)

In [ ]:
path ='/kaggle/working/ecg-sensor-data/ecg.csv'
import csv

In [ ]:
ecg_signal=[]
index=0
for i in ecg:
    if i>2:
        ecg_signal.append(ecg[index-100:index+100])
        index=index+1
   

In [ ]:
# plt.plot(ecg_signal[50])

In [ ]:
sig = ecg_signal[50]
pad = [sig[199]]*80
sig = pad + sig 
sig = sig + pad
sig = np.reshape(sig,(360,1))

In [ ]:
plt.plot(sig)

In [ ]:
list=[]
list.append(sig)
list = np.array(list)
list.shape
y_pred=model.predict(list)

In [ ]:
res = np.argmax(y_pred)
classes[res]

In [ ]:
sig = ecg_signal[55]
pad = [sig[199]]*80
sig = pad + sig 
sig = sig + pad
sig = np.reshape(sig,(360,1))
plt.plot(sig)
list=[]
list.append(sig)
list = np.array(list)
y_pred=model.predict(list)

In [ ]:
res = np.argmax(y_pred)
classes[res]

In [ ]:
sensor_pred=989*[0]+11*[3]

In [ ]:
type(sensor_pred)

In [ ]:
sensor_pred = pd.DataFrame(sensor_pred)

In [ ]:
per_class = sensor_pred[sensor_pred.shape[1]-1].value_counts()

In [ ]:
# plt.figure(figsize=(30,10))
# my_circle=plt.Circle( (0,0), 0.7, color='white')
# plt.pie(per_class, labels=['N', 'A'], colors=['#2a9d8f','#d62828'],autopct='%1.2f%%',textprops={'fontsize':11})
# p=plt.gcf()
# p.gca().add_artist(my_circle)
# plt.show()
# p.savefig('sensor_pred_2')

In [ ]:
sig = ecg_signal[56]
pad = [sig[199]]*80
sig = pad + sig 
sig = sig + pad
w = pywt.Wavelet('sym4')
maxlev = pywt.dwt_max_level(len(sig), w.dec_len)
coeffs = pywt.wavedec(sig, 'sym4', level=maxlev)
print(len(coeffs[5]))

In [ ]:
model.save('/kaggle/working/dpm.h5')